In [1]:
%matplotlib inline
# Load all necessary packages
# import sys
# sys.path.append("../")
import numpy as np
from tqdm import tqdm

In [3]:
from aif360.datasets import BinaryLabelDataset

In [4]:
from aif360.datasets import CompasDataset

In [5]:
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.algorithms.preprocessing.reweighing import Reweighing
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
        import load_preproc_data_adult, load_preproc_data_german, load_preproc_data_compas
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

In [6]:
# Load and preprocess the COMPAS dataset
dataset_race = load_preproc_data_compas(['race'])

In [7]:
privileged_groups = [{'race': 1}]
unprivileged_groups = [{'race': 0}]

<h3>1. Take the dataset with "race" as the protected attribute</h3>

In [8]:
protected_attribute_names=['race']

<h3>2. Split the dataset into training and test set</h3>

In [9]:
race_train, race_test = dataset_race.split([0.7], shuffle=True)

dataset_race_train, dataset_race_vt = dataset_orig.split([0.7], shuffle=True)
dataset_race_valid, dataset_race_test = dataset_orig_vt.split([0.5], shuffle=True)

<h3>3. Fix the bias for "race" using reweighing technique in the training set</h3>

In [11]:
from aif360.algorithms.preprocessing import Reweighing
RW = Reweighing(unprivileged_groups=[{'race': 0}], privileged_groups=[{'race': 1}])
compas_dataset_rw = RW.fit_transform(race_train)

<h4>Define Protected Attributes and Measure Initial Bias</h4> 

In [12]:
from aif360.metrics import BinaryLabelDatasetMetric
metric_orig_compas_race = BinaryLabelDatasetMetric(dataset_race, 
                                                   unprivileged_groups=[{'race': 0}], 
                                                   privileged_groups=[{'race': 1}])
print("Original Disparate Impact (Race):", metric_orig_compas_race.disparate_impact())
print("Original Statistical Parity Difference (Race):", metric_orig_compas_race.statistical_parity_difference())

Original Disparate Impact (Race): 0.7828387025392317
Original Statistical Parity Difference (Race): -0.13227942084985456


<h3>Metric with the transformed training data (race)</h3>

In [13]:
metric_orig_compas_race = BinaryLabelDatasetMetric(dataset_race, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_compas_race.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.132279


<h3>Train classifier on Race Dataset</h3>

In [15]:
# Logistic regression classifier and predictions
scale_orig = StandardScaler()
X_train = scale_orig.fit_transform(race_train.features)
y_train = race_train.labels.ravel()
w_train = race_train.instance_weights.ravel()

lmod = LogisticRegression()
lmod.fit(X_train, y_train, 
         sample_weight=race_train.instance_weights)
y_train_pred = lmod.predict(X_train)

# positive class index
pos_ind = np.where(lmod.classes_ == race_train.favorable_label)[0][0]

dataset_race_train_pred = race_train.copy()
dataset_race_train_pred.labels = y_train_pred

In [ ]:
dataset_race_test_pred = race_test.copy(deepcopy=True)
X_valid = scale_orig.transform(dataset_race_test_pred.features)
y_valid = dataset_race_test_pred.labels
dataset_race_test_pred.scores = lmod.predict_proba(X_valid)[:,pos_ind].reshape(-1,1)
print(dataset_race_test_pred.scores[:10])

dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
X_test = scale_orig.transform(dataset_orig_test_pred.features)
y_test = dataset_orig_test_pred.labels
dataset_orig_test_pred.scores = lmod.predict_proba(X_test)[:,pos_ind].reshape(-1,1)

In [ ]:
#if you want to see the dataset run this
print(compas_dataset_rw)

In [14]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_race.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_race.favorable_label, dataset_race.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_race.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_race.privileged_protected_attributes, 
      dataset_race.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_race.feature_names)

#### Training Dataset shape

(5278, 10)


#### Favorable and unfavorable labels

0.0 1.0


#### Protected attribute names

['race']


#### Privileged and unprivileged protected attribute values

[array([1.])] [array([0.])]


#### Dataset feature names

['sex', 'race', 'age_cat=25 to 45', 'age_cat=Greater than 45', 'age_cat=Less than 25', 'priors_count=0', 'priors_count=1 to 3', 'priors_count=More than 3', 'c_charge_degree=F', 'c_charge_degree=M']


<h3>4. Change the protected attribute to "sex" and repeat all steps</h3>

In [15]:
dataset_sex = load_preproc_data_compas(['sex'])

In [16]:
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

In [17]:
protected_attribute_names=['sex']

In [18]:
train, test = dataset_sex.split([0.7], shuffle=True)

In [19]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_sex.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_sex.favorable_label, dataset_sex.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_sex.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_sex.privileged_protected_attributes, 
      dataset_sex.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_sex.feature_names)

#### Training Dataset shape

(5278, 10)


#### Favorable and unfavorable labels

0.0 1.0


#### Protected attribute names

['sex']


#### Privileged and unprivileged protected attribute values

[array([1.])] [array([0.])]


#### Dataset feature names

['sex', 'race', 'age_cat=25 to 45', 'age_cat=Greater than 45', 'age_cat=Less than 25', 'priors_count=0', 'priors_count=1 to 3', 'priors_count=More than 3', 'c_charge_degree=F', 'c_charge_degree=M']


In [20]:
from aif360.algorithms.preprocessing import Reweighing
RW = Reweighing(unprivileged_groups=[{'sex': 0}], privileged_groups=[{'sex': 1}])
compas_dataset_rw = RW.fit_transform(train)

In [21]:
from aif360.algorithms.preprocessing import Reweighing
RW = Reweighing(unprivileged_groups=[{'sex': 0}], privileged_groups=[{'sex': 1}])
compas_dataset_rw = RW.fit_transform(test)

In [22]:
from aif360.metrics import BinaryLabelDatasetMetric
metric_orig_compas_sex = BinaryLabelDatasetMetric(dataset_sex, 
                                                   unprivileged_groups=[{'sex': 0}], 
                                                   privileged_groups=[{'sex': 1}])
print("Original Disparate Impact (Sex):", metric_orig_compas_sex.disparate_impact())
print("Original Statistical Parity Difference (Sex):", metric_orig_compas_sex.statistical_parity_difference())

Original Disparate Impact (Sex): 0.788415280444698
Original Statistical Parity Difference (Sex): -0.1350366105406292


<h3>Metric with the transformed training data (sex)</h3>

In [23]:
metric_orig_compas_sex = BinaryLabelDatasetMetric(dataset_sex, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
display(Markdown("#### Transformed training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_compas_sex.mean_difference())

#### Transformed training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.135037


In [ ]:
#if you want to see the dataset run this
print(compas_dataset_rw)

<h3>Train a logistic regression classifier for the new transformed data</h3>

In [ ]:
# Logistic regression classifier and predictions
scale_orig = StandardScaler()
X_train = scale_orig.fit_transform(dataset_orig_train.features)
y_train = dataset_orig_train.labels.ravel()
w_train = dataset_orig_train.instance_weights.ravel()

lmod = LogisticRegression()
lmod.fit(X_train, y_train, 
         sample_weight=dataset_orig_train.instance_weights)
y_train_pred = lmod.predict(X_train)

# positive class index
pos_ind = np.where(lmod.classes_ == dataset_orig_train.favorable_label)[0][0]

dataset_orig_train_pred = dataset_orig_train.copy()
dataset_orig_train_pred.labels = y_train_pred